# Reddit-Mining
## Die Ziele im Projekt

Allgemein ist die Aufgabe Texte systematisch aus dem Web zu gewinnen und diese dann auszuwerten.

Wir haben uns zur Aufgabe gemacht durch diverse Reddit-Spiele-Foren (https://reddit.com) zu crawlen und verschiedene Aussagen über die Nutzer und die Spiele der einzelnen Foren zu machen:
    - Wie freundlich sind die Spieler der einzelnen Foren?
    - In welchen Spielen wird am meisten im Web diskutiert?
    
Zusätzlich ist es das Ziel, einen beliebigen Text zu einem der Spiele zuzuordnen.

Zur Datenspeicherung wird nach Aufgabe eine SQL-Lite Datenbank verwendet.   

Wichtig für uns ist die modularität, sodass wir schnell weitere Reddit-Foren hinzufügen können.

# Scraping

Dieser Abschnitt behandelt das gesamte Scraping der einzelnen Reddit Pages.

Zunächst werden die Spieleforen, die benutzt werden, angegegeben:

In [68]:
games= {}
games["dota2"] = 'https://www.reddit.com/r/DotA2/'
games["csgo"] = 'https://www.reddit.com/r/GlobalOffensive/'
#games["spacex"] = 'https://www.reddit.com/r/spacex/' #<- War lediglich aus interesse
#games["lol"] = 'https://www.reddit.com/r/leagueoflegends/'
#games["darksouls3"] = 'https://www.reddit.com/r/darksouls3/'
#games["witcher3"] = 'https://www.reddit.com/r/Witcher3/'
#games["smite"] = 'https://www.reddit.com/r/Smite/'
#games["aoe4"] = 'https://www.reddit.com/r/aoe4/'
#games["unrealtournament"] = 'https://www.reddit.com/r/unrealtournament/'
#games["battlefield1"] = 'https://www.reddit.com/r/battlefield_one/'

Als nächstes werden Parameter, welche beispielsweise die Datenmenge bestimmen, initialisiert und definiert:

In [69]:
pages_per_forum = 3

Nun kann man eigentlich mit dem "scrapen" beginnen. Hierfür wird "scrapy" verwendet, der code orientiert sich an den Aufgabenblättern der Vorlesung.

In [70]:
import scrapy
import requests
#copy of sheet 05
def gen_scrapy_response(url):
    # define user agent to simulate interactive user
    user_agent = 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    req = requests.get(url, headers={ "user-agent": user_agent })
    return scrapy.http.TextResponse(req.url, body=req.text, encoding='utf-8')

In der Analyse ist uns aufgefallen, dass alle Reddit-Foren zwar unterschiedliche Designs haben, aber immer die selbe Struktur (auch in Bezug auf den HTML-Baum und die CSS-Klassen) besitzen. Aus diesem Grund können wir das selbe vorgehen auf alle Foren anwenden.

Zunächst werden alle (oder so viele wie möglich) Seiten aus den einzelnen Foren ausgelesen:

In [71]:
reddit_pages = {}
#extracts all reddit overview pages for each game analyzed
for game in games:
    link = games[game]
    #predefine an empty list
    reddit_pages[game] = []
    for i in range(pages_per_forum-1):
        x = gen_scrapy_response(link)
        reddit_pages[game].append(x)
        link = str(x.css("span.next-button > a::attr(href)").extract_first()) #next button, missing if no other page exists
        if(link == "None"): ##cancel if no other page exists
            break

Nun werden alle Threads auf den gegeben Seiten erfasst, in diesen steht der großteil der Daten (wie die einzelnen Texte). Diese werden in Listen gespeichert, welche wieder in einem Dictionary, sortiert nach den Foren, verwaltet werden:

In [72]:
reddit_threads = {}
#extracts all redit threads from the pages available
for game in reddit_pages:
    #predefine an empty list
    reddit_threads[game] = []
    #iterate through all available pages per game
    for page in reddit_pages[game]:
        thread_links = page.css("a.comments::attr(href)").extract() #comment button
        reddit_threads[game].extend(thread_links)
#cleaning old variables
#del reddit_pages

Nachdem alle Links erfasst wurden, kann man durch diese Listen iterieren um die Daten zu erfassen. Die Daten werden in einer Liste aus Dictionaries gespeichert, diese dcits besitzen zunächst folgende Informationen:
- game: Das Spiel für welches dieser Kommentar geschrieben wurde
- thread: Der Titel des Threads, für welchen diese Kommentar geschrieben wurde
- user: Der Benutzer der diesen Kommentar geschrieben hat
- content: Der Text-Inhalt des Kommentars

Um gleich die Daten etwas aufzubereiten werden Regular Expressions verwendet

In [99]:
import regex as re
all_comments = []
#iterating through the games
for game in reddit_threads:
    #creating the new dictionary
    comment = {}
    #iterating through the threads of a game
    for thread in reddit_threads[game]:
        response = gen_scrapy_response(thread)
        title = response.css("a.title::text").extract_first()
        comments = response.css("div.comment")
        for comment in comments:
            user = comment.css("div.entry > p.tagline > a.author::text").extract_first()
            text = " ".join(comment.css("div.usertext-body > div.md > *::text").extract())
            text = re.sub('\s+',' ',text)
            all_comments.append({"game":game,"thread":title,"user":user,"content":text})

## Datenbank

Für die weitere Analyse werden die Daten in einer relationalen SQL-Lite Datenbank gespeichert

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["figure.dpi"] = 100

def bar_plot(list_of_tuples, columns=('Wort', 'Anzahl'), title=None): 

    labels = [t[0] for t in list_of_tuples]
    values = [t[1] for t in list_of_tuples]
    dummy_pos = np.arange(len(labels))
    
    # Erstellung des Balkendiagramms
    fig=plt.figure(figsize=(18, 10))
    plt.bar(dummy_pos, values)
    plt.xticks(dummy_pos, labels,  rotation="vertical", fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlabel(columns[0], fontsize=20)
    plt.ylabel(columns[1], fontsize=20)
    if title: 
        plt.title(title, fontsize=20)